Currently the atlas maps are not in the form we want, so we prep them here:

1. V1 atlas, restricted to 4-10 degrees
2. LGN atlas, combine L and R

In [ ]:
# Import necessary libraries
import statsmodels.api as sm
from statsmodels.stats.anova import AnovaRM
import pingouin as pg
import os
from joblib import dump, load
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn import image
from nilearn.reporting import make_glm_report
from nilearn.image import load_img, threshold_img
from nilearn.glm.contrasts import compute_contrast
from nilearn.input_data import NiftiMasker
from nilearn.plotting import view_img
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.second_level import SecondLevelModel
from nilearn.maskers import NiftiMasker, NiftiLabelsMasker
from nilearn.plotting import view_img_on_surf
import pandas as pd
from collections import defaultdict
import json, joblib
import pandas as pd
from nilearn.image import mean_img
from nilearn.glm.first_level import FirstLevelModel
from nilearn.masking import apply_mask
import seaborn as sns

print('Done importing libraries')

In [ ]:
# Set up

# directories
base_dir = '/scratch/groups/Projects/P1490'
data_dir = 'Data' # directory of all participants' data
t1_dir='freesurfer' # directory where recon-all T1 is


In [ ]:
# The benson atlas gives us an area map and an eccentricity map.
# We want one atlas for all ROIs, only in the eccentricity 4-10 degrees.

ecc_map = nib.load(os.path.join(base_dir,t1_dir,'MNI/mri/tempBens/benson14_eccen.nii.gz'))
ecc_data = ecc_map.get_fdata()

ecc_data[(ecc_data <= 4) | (ecc_data >= 10)] = 0

ecc_map_restricted = nib.Nifti1Image(ecc_data, ecc_map.affine, ecc_map.header)
nib.save(ecc_map_restricted, os.path.join(base_dir,t1_dir,'MNI/mri/tempBens/benson14_eccen_4_10.nii.gz'))

area_map = nib.load(os.path.join(base_dir,t1_dir,'MNI/mri/tempBens/benson14_varea.nii.gz'))
varea_data = area_map.get_fdata()

# mask is True wherever eccen equals 0
mask = (ecc_data == 0)

combined = varea_data.copy()
combined[mask] = 0.0          
combined = combined.astype(int)

restricted_map = nib.Nifti1Image(combined, affine=area_map.affine, header=area_map.header)
nib.save(restricted_map, os.path.join(base_dir,t1_dir,'MNI/mri/tempBens/benson14_eccen_4_10_varea.nii.gz'))


In [ ]:
view=view_img(restricted_map, black_bg=True, threshold=0)
view

In [ ]:
# Similar to above, but we just want to look at V1, not all ROIs

restricted_map = nib.load(os.path.join(base_dir,t1_dir,'MNI/mri/tempBens/benson14_eccen_4_10_varea.nii.gz'))
restricted_data = restricted_map.get_fdata()

# mask is True wherever eccen equals 0
mask = (restricted_data != 1)

v1_data = restricted_data.copy()
v1_data[mask] = 0.0            # or use 0 if dtype is integer
v1_data = np.around(v1_data)

v1_img = nib.Nifti1Image(v1_data, affine=restricted_map.affine, header=restricted_map.header)
nib.save(v1_img, os.path.join(base_dir,t1_dir,'MNI/mri/tempBens/benson14_eccen_4_10_v1.nii.gz'))

view=view_img(v1_img, black_bg=True, symmetric_cmap=False)
view

In [ ]:
# Plot V1 ROI on surface

from nilearn.plotting import view_img_on_surf
from matplotlib.colors import LinearSegmentedColormap

solid_color = LinearSegmentedColormap.from_list("solid_yellow", ["yellow", "yellow"])

view = view_img_on_surf(v1_img, threshold=0.01, surf_mesh='fsaverage7', 
                        symmetric_cmap=False, cmap=solid_color, colorbar=False,
                        black_bg=True, bg_on_data=True,
                        vol_to_surf_kwargs=dict(
                        radius=2.0, # neighbourhood (mm)
                        interpolation="nearest", # 'linear' or 'nearest'
                        kind="ball", # 'auto', 'line', 'ball', or 'depth'
                        n_samples=10 # number of samples to average
                    )) # Change fsaverage5 to fsaverage if you want to use the higher-res fsaverage mesh

view 

In [ ]:
# The LGN atlas has seperate L and R, so we just combine them into one map.

lgn_left_map = nib.load(os.path.join(base_dir,data_dir,'LGN_atlas/left/9-LGN.nii.gz'))
lgn_left_data = lgn_left_map.get_fdata()

lgn_right_map = nib.load(os.path.join(base_dir,data_dir,'LGN_atlas/right/9-LGN.nii.gz'))
lgn_right_data = lgn_right_map.get_fdata()

lgn_combined_data = lgn_left_data + lgn_right_data
combined_img = nib.Nifti1Image(lgn_combined_data, affine=lgn_left_map.affine, header=lgn_left_map.header)
nib.save(combined_img, os.path.join(base_dir,data_dir,'LGN_atlas/9-LGN.nii.gz'))
